In [1]:

import os
import pandas as pd
from PIL import Image
import tf_keras as keras
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split



2025-06-04 19:42:04.673075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749055324.689922    8553 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749055324.694490    8553 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749055324.706948    8553 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749055324.706968    8553 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749055324.706969    8553 computation_placer.cc:177] computation placer alr

In [2]:
LABEL_GROUPS = {
    "age_gender": ['Female', 'AgeOver60', 'Age18-60', 'AgeLess18'],
    "view": ['Front', 'Side', 'Back'],
    "accessories": ['Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack', 'HoldObjectsInFront'],
    "upper_clothing": ['ShortSleeve', 'LongSleeve', 'UpperStride', 'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LongCoat'],
    "lower_clothing": ['LowerStripe', 'LowerPattern', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots'],
}

class ImageDatasetTF:
    def __init__(self, dataframe: pd.DataFrame, img_folder: str, label_group: list, image_size=(128, 256), transform=None):
        self.data = dataframe
        self.img_folder = img_folder
        self.image_size = image_size
        self.transform = transform

        self.label_columns = label_group
        self.data = self.data[["Image"] + self.label_columns]  # Keep only relevant columns

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]["Image"]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB').resize(self.image_size)

        if self.transform:
            image = self.transform(image)
        else:
            image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]

        label = self.data.iloc[idx][self.label_columns].values.astype(np.float32)
        return image, label

    def generator(self):
        for i in range(len(self)):
            yield self[i]


In [3]:
train=pd.read_csv("PA-100K/train.csv")
validate= pd.read_csv("PA-100K/val.csv")
test=pd.read_csv("PA-100K/test.csv")

age_gender_labels = LABEL_GROUPS["age_gender"]
upper_clothing_labels = LABEL_GROUPS["upper_clothing"]


age_train = ImageDatasetTF(train,"PA-100K/data", age_gender_labels)
age_validation = ImageDatasetTF(validate,"PA-100K/data", age_gender_labels)
age_test= ImageDatasetTF(test,"PA-100K/data/", age_gender_labels)


upper_clothing_train = ImageDatasetTF(train, "PA-100K/data", upper_clothing_labels)
upper_clothing_validate= ImageDatasetTF(train, "PA-100K/data", upper_clothing_labels)
upper_clothing_test= ImageDatasetTF(train, "PA-100K/data", upper_clothing_labels)





In [4]:


input_t=keras.Input(shape=(256,128,3))
efficient_model=keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False,weights='imagenet',input_tensor=input_t)

I0000 00:00:1749055343.051317    8553 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4149 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:

for layer in efficient_model.layers[:390]:
  layer.trainable=False
model=keras.models.Sequential()
model.add(efficient_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(4,activation='sigmoid'))

In [7]:
optimizer= keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.fit(age_train, epochs=50 )

ValueError: Failed to find data adapter that can handle input: <class '__main__.ImageDatasetTF'>, <class 'NoneType'>

In [ ]:
res_model=keras.applications.ResNet152V2(include_top=False, input_tensor=input_t)
for layer in res_model.layers[:-14]:
    layer.trainable=False
optimizer = keras.optimizers.Adam(learning_rate=3e-3)
model2 = keras.models.Sequential([
    res_model,
    keras.layers.GlobalAveragePooling2D(),  # Critical!
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(26, activation='sigmoid')
   
])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model2.compile(optimizer=optimizer,loss=keras.losses.BinaryCrossentropy(),  metrics=['binary_accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()])
model.fit(train_dataset,validation_data= test_dataset, epochs=50)

Epoch 1/50
   1250/Unknown - 315s 252ms/step - loss: 0.3196 - accuracy: 0.4779

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True
)


In [2]:

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))


2.19.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Num GPUs Available: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


RuntimeError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [4]:
df= pd.read_csv("PA-100K/train.csv")
print(df.columns)


Index(['Image', 'Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front',
       'Side', 'Back', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack',
       'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'UpperStride',
       'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern',
       'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots'],
      dtype='object')
